# Download dataset and code

In [1]:
!mkdir /tmp/data/ -p
!gdown --id '18WeCp7OPGulOgMl-KN6DN1CJw47IlTQ2' --output /tmp/data/lte.train.csv
!gdown --id '18_nW3nufgDzFFmpuPAWyKUbO11gLU8Ud' --output /tmp/data/lte.test.csv
!git clone https://github.com/lirits/NetworkTrafficPrediction.git
!ls /tmp/data/

Downloading...
From: https://drive.google.com/uc?id=18WeCp7OPGulOgMl-KN6DN1CJw47IlTQ2
To: /tmp/data/lte.train.csv
100% 20.0M/20.0M [00:00<00:00, 63.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=18_nW3nufgDzFFmpuPAWyKUbO11gLU8Ud
To: /tmp/data/lte.test.csv
100% 396k/396k [00:00<00:00, 3.40MB/s]
Cloning into 'NetworkTrafficPrediction'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 137 (delta 72), reused 102 (delta 39), pack-reused 0
Receiving objects: 100% (137/137), 550.92 KiB | 2.21 MiB/s, done.
Resolving deltas: 100% (72/72), done.
lte.test.csv  lte.train.csv


In [2]:
import sys
if not 'NetworkTrafficPrediction' in sys.path:
    sys.path += ['NetworkTrafficPrediction']

In [ ]:
!pip3 install wandb

In [5]:
import torch 
from torch.utils.data import DataLoader
from torch import nn
import matplotlib.pyplot as plt
import torch.optim as optim
from data.custom_dataset import LteTrafficDataset
import torch.nn.functional as F
# from utils.tools import train_loop,test_loop
# from model.LSTM import LSTM
from model.Transformer import TransformerModule
from utils.radam import RAdam
import wandb
wandb.init(project='Network-Traffic-Prediction')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Parameters
train_file_path = '/tmp/data/lte.train.csv'
test_file_path = '/tmp/data/lte.test.csv'
CellName = 'Cell_003781'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform,target_transform = True,True
# Hyperparameters of Dataset
wandb.config.batch_size = 64
wandb.config.windows_size = 128
wandb.config.target_size = 32
wandb.config.transform,wandb.config.target_transform = True,True
wandb.config.learning_rate = 1e-4
wandb.config.epochs= 50

In [7]:
# training_data = LteTrafficDataset(train_file_path,CellName=CellName,windows_size=windows_size,target_size=target_size,transform=transform,target_transform=target_transform)
# train_dataloader = DataLoader(training_data,batch_size=batch_size,shuffle=True)
# test_data = LteTrafficDataset(test_file_path,CellName,windows_size,target_size,transform,target_transform)
# test_dataloader = DataLoader(test_data,batch_size=batch_size,shuffle=False)
training_data = LteTrafficDataset(train_file_path,CellName=CellName,windows_size=wandb.config.windows_size,target_size=wandb.config.target_size,transform=wandb.config.transform,target_transform=wandb.config.target_transform)
train_dataloader = DataLoader(training_data,batch_size=wandb.config.batch_size,shuffle=True,drop_last=True)
test_data = LteTrafficDataset(test_file_path,CellName,wandb.config.windows_size,wandb.config.target_size,wandb.config.transform,wandb.config.target_transform)
test_dataloader = DataLoader(test_data,batch_size=wandb.config.batch_size,shuffle=False,drop_last=True)

In [31]:
# # Hyperparameters of Transformer
# input_dim = 1
# output_dim = int(target_size/2) # input_size of decoder
# embed_dim = 1024 # ValueEmbedding
# nhead = 8
# dim_hid = 1024 #the dimension of the feedforward network model
# num_encoder = 3
# num_decoder = 1 
# dropout = 0.05
# pred_size = target_size
# activation = 'gelu'
# wandb.config.batch_size = 64
# wandb.config.windows_size = 128
# wandb.config.target_size = 32
# wandb.config.transform,wandb.config.target_transform = True,True
# wandb.config.learning_rate = 1e-4
# wandb.config.epochs= 50

import math
sweep_config = {'method': 'random'}
transform,target_transform = True,True

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }
sweep_config['metric'] = metric

parameters_dict = {
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
    },
    'windows_size' :{
        "values":[64,80,96]
    },
    'target_size':{
        'values':[32,48,64]
    },
    'learning_rate' :{
        'values':[1e-1,1e-2,1e-3,1e-4,1e-5]
    },
    'epochs':{
        'value':200
    }
}
sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'optimizer':{
        'values': ['adam','sgd','radam']
    },
    'dropout':{
        'values': [0.05,0.1,0.15,0.5]
    },
    'input_dim':{
        'value': 1
    },
    'output_dim':{
        "min":0,
        "max":32
    },
    "embed_dim":{
        'values':[256,512,1024,1536,2048]
    },
    "nhead":{
        'values':[1,2,4,8,16]
    },
    "dim_hid":{
        'values':[256,512,1024,1536,2048]
    },
    "num_encoder":{
        'values': [1,2,3,4,5]
    },
    "num_decoder":{
        'values':[1,2,3,4,5]
    },
    "activation":{
        'values':['relu','gelu']
    },

    })

In [32]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'activation': {'values': ['relu', 'gelu']},
                'batch_size': {'distribution': 'q_log_uniform',
                               'max': 5.545177444479562,
                               'min': 3.4657359027997265,
                               'q': 1},
                'dim_hid': {'values': [256, 512, 1024, 1536, 2048]},
                'dropout': {'values': [0.05, 0.1, 0.15, 0.5]},
                'embed_dim': {'values': [256, 512, 1024, 1536, 2048]},
                'epochs': {'value': 200},
                'input_dim': {'value': 1},
                'learning_rate': {'values': [0.1, 0.01, 0.001, 0.0001, 1e-05]},
                'nhead': {'values': [1, 2, 4, 8, 16]},
                'num_decoder': {'values': [1, 2, 3, 4, 5]},
                'num_encoder': {'values': [1, 2, 3, 4, 5]},
                'optimizer': {'values': ['adam', 'sgd', 'radam']},
                'output_dim': {'max': 32,

In [35]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: sp19xi9s
Sweep URL: https://wandb.ai/lirit/pytorch-sweeps-demo/sweeps/sp19xi9s


In [46]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(train_file_path,CellName,config.windows_size,config.target_size,config.batch_size)
        network = build_network(config.input_dim,config.output_dim,config.embed_dim,config.nhead,config.dim_hid,config.num_encoder,config.num_decoder,config.dropout,config.target_size,config.activation)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})  

def build_dataset(train_file_path,CellName,windows_size,target_size,batch_size):
    dataset = LteTrafficDataset(train_file_path,CellName=CellName,windows_size=windows_size,target_size=target_size)
    loader = DataLoader(dataset,batch_size=batch_size,shuffle=True,drop_last=True)
    return loader


def build_network(input_dim,output_dim,embed_dim,nhead,dim_hid,num_encoder,num_decoder,dropout,pred_size,activation):
    network = TransformerModule(input_dim,output_dim,embed_dim,nhead,dim_hid,num_encoder,num_decoder,dropout,pred_size,activation)
    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    elif optimizer == 'radam':
        optimizer = RAdam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.mse_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

In [47]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: mao82zk2 with config:
wandb: 	activation: relu
wandb: 	batch_size: 47
wandb: 	dim_hid: 512
wandb: 	dropout: 0.5
wandb: 	embed_dim: 1024
wandb: 	epochs: 200
wandb: 	input_dim: 1
wandb: 	learning_rate: 1e-05
wandb: 	nhead: 8
wandb: 	num_decoder: 5
wandb: 	num_encoder: 3
wandb: 	optimizer: sgd
wandb: 	output_dim: 15
wandb: 	pred_size: 32
wandb: 	target_size: 32
wandb: 	windows_size: 64


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
!nvidia-smi

Thu Oct  7 05:26:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------